In [1]:
import random
import math
import csv
import time
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from multiprocessing import Pool
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import accuracy_score,top_k_accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.calibration import CalibratedClassifierCV
from sklearn.multioutput import MultiOutputRegressor
from joblib import dump, load
from helper_func import analysis_func_expnum,analysis_func_layers,analysis_func_noise
from functools import partial

### Test effect of number of simulation on MAE, MSE and training time

In [2]:
num_exprs=[100,250,500,1000,2500,5000,10000,25000]#,50000,100000]
noise=0.01

#Define final MLP
model_single = MLPRegressor(random_state=1, max_iter=500,tol=0.001,early_stopping=True,activation='relu',alpha=0.000075,
               hidden_layer_sizes=(2048, 1024, 512, 256, 128, 64, 32),solver='adam',beta_1=0.5,beta_2=0.05,learning_rate='constant')
model = MultiOutputRegressor(model_single)
    

In [3]:
analysis_partial=partial(analysis_func_expnum,model=model,noise=noise)
p = Pool(processes=len(num_exprs))
results = p.map(analysis_partial, num_exprs)
p.close()
print(results)


[(100, 90.21399426460266, 0.3120002746582031, 3.840708363519858, 0.008091961266977022, 1.5350019648838806, 0.08214422053279315), (250, 95.4999942779541, 0.42600178718566895, 1.4686882058669681, 0.005577513460909925, 0.8637725907842976, 0.06781801345814259), (500, 154.82499718666077, 0.030004262924194336, 1.4639984924021587, 0.005157083442649115, 0.9211604720317359, 0.06326837914735595), (1000, 301.61798191070557, 0.29600048065185547, 2.189523626198654, 0.004423549861623553, 1.08339327491756, 0.0546225584579277), (2500, 682.7070150375366, 0.3339827060699463, 1.149977351827786, 0.0043971717621779085, 0.8624305855970766, 0.05643051971092171), (5000, 1278.0830030441284, 0.4999966621398926, 1.0800110725083458, 0.004817148847047539, 0.8043187847794855, 0.058366254067518095), (10000, 2122.258367061615, 0.3820023536682129, 0.8571869928383723, 0.00445992178893079, 0.704767808587045, 0.05635085981372063), (25000, 2839.309496164322, 0.6310043334960938, 0.8559531805481442, 0.004458043019043553, 0.

In [4]:
#Save results
dump(results, 'mlp_number_of_sims_results.joblib')

['number_of_sims_results.joblib']

### Test effect of MLP complexity (number of layers number of nodes) to MAE, MSE,training time and prediction time

In [4]:
num_exprs=1000
noise=0.01
layers=[(64, 32),(128, 64, 32),(256, 128, 64, 32),(512, 256, 128, 64, 32),(1024, 512, 256, 128, 64, 32),(2048, 1024, 512, 256, 128, 64, 32)]


In [5]:
analysis_partial=partial(analysis_func_layers,num_exprs=num_exprs,noise=noise)
p = Pool(processes=len(layers))
results = p.map(analysis_partial, layers)
p.close()
print(results)

[((64, 32), 1.487992286682129, 0.0019996166229248047, 1.2243842844504758, 0.0055577261925499775, 0.8773359380921789, 0.06126102512074616), ((128, 64, 32), 7.5799970626831055, 0.003000497817993164, 1.2648996591459716, 0.004653656131532471, 0.8420268717892138, 0.05672162161632187), ((256, 128, 64, 32), 6.363000154495239, 0.03200054168701172, 1.328101329482412, 0.004806048454813376, 0.9365440487920275, 0.057418885798779594), ((512, 256, 128, 64, 32), 8.123998403549194, 0.009004592895507812, 1.205520470304328, 0.0046948876458005, 0.8953334080136652, 0.05814578278036106), ((1024, 512, 256, 128, 64, 32), 29.39205265045166, 0.015000581741333008, 1.780358211693911, 0.004717325121433077, 0.9934192292061604, 0.05713418878011879), ((2048, 1024, 512, 256, 128, 64, 32), 69.44904971122742, 0.024001121520996094, 2.189523626198654, 0.004423549861623553, 1.08339327491756, 0.0546225584579277)]


In [6]:
#Save results
dump(results, 'mlp_number_of_layers_results.joblib')

['number_of_layers_results.joblib']

### Test effect of signal's noise on MAE and MSE

In [2]:
num_exprs=4000
noise=[0.001,0.0025,0.005,0.01,0.025,0.05,0.1,0.25,0.5]
#Define final MLP
model_single = MLPRegressor(random_state=1, max_iter=500,tol=0.001,early_stopping=True,activation='relu',alpha=0.000075,
               hidden_layer_sizes=(512, 256, 128, 64, 32),solver='adam',beta_1=0.5,beta_2=0.05,learning_rate='constant')
model = MultiOutputRegressor(model_single)

In [4]:
analysis_partial=partial(analysis_func_noise,model=model,exp=num_exprs)
p = Pool(processes=len(noise))
results = p.map(analysis_partial, noise)
p.close()
print(results)

[(0.001, 94.56650876998901, 0.025005340576171875, 0.946622800257375, 0.0036980783616163905, 0.7681822671523955, 0.049729149680072006), (0.0025, 88.39050149917603, 0.044005393981933594, 0.8153880066181444, 0.0037294308982114273, 0.6922227018923826, 0.05038773124538711), (0.005, 82.204509973526, 0.03899264335632324, 0.9862688601575341, 0.004691925101690524, 0.7873210538200013, 0.05775529698099843), (0.01, 95.5204963684082, 0.016999483108520508, 1.0239386981459895, 0.004227878540673349, 0.7673781410688442, 0.05451399523546472), (0.025, 83.11550498008728, 0.036002397537231445, 1.2334875946484423, 0.004826353074998528, 0.8747723443403541, 0.05955306967759899), (0.05, 88.55051159858704, 0.031002283096313477, 1.474366037968677, 0.005529943555415876, 0.947105152546342, 0.06456359379014172), (0.1, 89.81050825119019, 0.026003360748291016, 1.7973716889350926, 0.004572053866777632, 1.052911904177955, 0.05743902504794387), (0.25, 98.9875078201294, 0.015000343322753906, 2.0863101864878315, 0.0056043

In [5]:
#Save results
dump(results, 'mlp_noise_results.joblib')

['mlp_noise_results.joblib']